# Dataset, DataLoader
---

### 1. Dezero에서의 Dataset, DataLodaer
---
Daatset을 설정한 뒤에 DataLoder로 설정한 Dataset을 끌고 온다.

Dataset 은 샘플과 정답(label)을 저장하고, DataLoader 는 Dataset 을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체(iterable)로 감싼다.

> PyTorch의 도메인 특화 라이브러리들은 (FashionMNIST와 같은) 미리 준비해둔(pre-loaded) 다양한 데이터셋을 제공합니다.    
> 데이터셋은 torch.utils.data.Dataset 의 하위 클래스로 개별 데이터를 특정하는 함수가 구현되어 있습니다.    
> 이러한 데이터셋은 모델을 만들어보고(prototype) 성능을 측정(benchmark)하는데 사용할 수 있습니다.    
> 1. 비디오 데이터셋 - https://pytorch.org/vision/stable/datasets.html  
> 2. 텍스트 데이터셋 - https://pytorch.org/text/stable/datasets.html  
> 3. 오디오 데이터셋 - https://pytorch.org/audio/stable/datasets.html  


Dataset은 왜 만들었을까?

데이터가 100만개라면, 거대한 데이터를 하나의 ndarray 인스턴스로 처리하면 모든 원소를 한꺼번에 메모리에 올려야 한다. 

당연히 메모리는 부담스러운 상황

그렇기 때문에 이러한 문제에 대응이 가능하도록 Dataset 클래스를 만들었다.


### 2. Dezero의 Dataset(복습)
---

In [2]:
import numpy as np

class Dataset: # Dezero의 Dataset
    def __init__(self, train=True, transform=None, target_transform=None):
        self.train = train
        self.transform = transform
        self.target_transform = target_transform

        if self.transform is None:
            self.transform = lambda x: x
        if self.target_transform is None:
            self.target_transform = lambda x: x

        self.data = None # 입력 데이터 보관
        self.label = None # 레이블 보관
        self.prepare()

    def __getitem__(self, index): # 객체에 슬라이싱이 가능하게 하는 특수 메소드 
        assert np.isscalar(index) # 만약에 받은 요소가 스칼라이면 실행
        if self.label is None: # 레이블이 존재치 않는다면
            return self.transform(self.data[index]), None # 값에 인덱싱해서 반환
        else:
            return self.transform(self.data[index]), self.target_transform(self.label[index])

    def __len__(self): # 데이터셋 길이 반환
        return len(self.data)

    def prepare(self): # 유도 쿨래스에서 마저 정의해야 한다. 
        pass

class Spiral(Dataset): # 이렇게 데이터셋에 각종 데이터를 가져올 수 있게끔 설정해 뒀을것이다. 
    def prepare(self): # 이런식으로 인스턴스 변수인 data와 label데이터를 설정한다
        self.data, self.label = get_spiral(self.train)

pytorch또한 위와 비슷한 형상을 띄고 있을것이다. 

여기서 중요한 것은 데이터셋은 이미 다 정의가 되어있다는 것

파이토치가 정해준 데이터셋을 그냥 이용하면 끝이다. 물론 튜닝을 할 수 있지만... 굳이?

### 3. pytorch의 dataset
---

기본적으로 제공되는 틀인 Dataset이라는 클래스가 존재한다.

이 클래스를 상속받고 

__init__(self) : 메서드는 객체를 생성할 때 실행되는 메서드, 즉 생성자입니다. 여기에는 모델에 사용할 데이터를 담아두는 등 어떤 인덱스가 주어졌을 때 반환할 수 있게 만드는 초기 작업을 수행합니다.

__len__(self) : 학습에 사용할 데이터의 총 개수라고 볼 수 있는데, 즉 얼마만큼의 인덱스를 사용할지를 반환하는 메서드입니다.

__getitem__(self, index) : 메서드는 어떤 인덱스가 주어졌을 때 해당되는 데이터를 반환하는 메서드입니다. numpy 배열이나 텐서 형식으로 반환합니다. 보통 입력과 출력을 튜플 형식으로 반환하게 됩니다.

들을 정의해주면 사용자 전용 Dataset 완성

찾아보니 __len__ 메서드와 __getitiem__ 메서드를 지닌 어떤 것도 죄다 Dataset이 될 수 있다고 한다. (물론 오류에 대한 책임은 작성자가 지어야 한다.)

> 사용자 정의 Dataset 클래스는 반드시 3개 함수를 구현해야 합니다: __init__, __len__, and __getitem__. 



In [3]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        pass
    def __getitem__(self, index):
        pass
    def __len__(self):
        pass

### 4. Dezero의 Dataloader
---


In [3]:
import math
import random
import numpy as np

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True, gpu=False):
        self.dataset = dataset # 데이터 셋
        self.batch_size = batch_size # 배치 크기
        self.shuffle = shuffle # 에포크별로 셔플
        self.data_size = len(dataset) # 데이터 사이즈 
        self.max_iter = math.ceil(self.data_size / batch_size) # 최대 반복수
        self.gpu = gpu # GPU

        self.reset()

    def reset(self):
        self.iteration = 0 # 반복자를 0으로 설정, 반복자는 데이터를 순차적으로 추출하는 기능을 제공한다.
        if self.shuffle: # 만약에 셔플할 생각이거든
            self.index = np.random.permutation(len(self.dataset)) # 그리 해주마
        else: # 아니거든
            self.index = np.arange(len(self.dataset)) #그냥 천천히 반환해라

    def __iter__(self): # 클래스를 파이썬 반복자로 사용하기 위해 넣은 특수 메서드 
        return self # 특수 메서드를 구현하고 자기 자신을 반환하도록 하는게 기본 문법.

    def __next__(self): # 반복자에서 데이터를 순서대로 추출하기 위한 메서드
        if self.iteration >= self.max_iter: #최대 반복자수보다 크거나 같다면
            self.reset() # 아 ㅋㅋ 리셋해야지 뭐
            raise StopIteration() # 다음 원소를 반환하도록 구현, 반환할 원소가 없으면 StopIteration()을 수행 

        i, batch_size = self.iteration, self.batch_size
        
        batch_index = self.index[i * batch_size:(i + 1) * batch_size]
        batch = [self.dataset[i] for i in batch_index]

        xp = cuda.cupy if self.gpu else np
        x = xp.array([example[0] for example in batch])
        t = xp.array([example[1] for example in batch])

        self.iteration += 1
        return x, t # 데이터와 레이블을 반환

    def next(self):
        return self.__next__()

Dataset과는 달리 DataLoader는 그닥 사용자가 수정할 이유가 없다.

DataLoader는 그저 Dataset를 기반으로 데이터를 꺼내는 것일뿐 그 이상의 기능을 할 이유가 없다.

### 5. pytorch의 DataLoader
---
정의는 다음과 같다.

DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)


parameter 설명

1. dataset는 필수 입력 인자다.

2. bath_size = 1 : 배치 크기이다.

3. shuffle = False : 에포크별로 데이터셋을 뒤섞을건지 여부를 결정하는 것

나머지는 많이 사용하지는 않는다. 그 외 설명은 다음 글을 참고하길 바란다.
> DataLoader parameter 별 용도 : https://subinium.github.io/pytorch-dataloader/

</br>
</br>
</br>

``` py
# DataLoader 사용 예제

from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

for epoch in range(max_epoch):
    for x, t in train_dataloader:
        print(x.shape, t.shape)

        ...
        
```

> https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader 데이터로더에 대한 자세한 파라미터 값

### 6. 직접 데이터셋 만들기
---


In [ ]:
class SimpleDataset(Dataset):
    def __init__(self, t):
        # 데이터셋을 처음 선언할 때, 즉 데이터셋 오브젝트가 생길 때 자동으로 불리는 함수이고, 여기에 우리가 몇 가지 인수들을 입력받도록 만들 수 있다 (path, transform 같은 것들).

    def __len__(self):
        # 데이터셋의 길이다. 만약 dataset을 선언하고 나서 len(어떤 dataset)을 하면 내부적으로는 이 len 함수가 불리는 것이다. 
        # 이 len은 나중에 데이터셋을 선언하고 데이터로더를 사용할 때 또 내부적으로 사용된다. 
        # (데이터셋의 len을 알아야 데이터로더가 미니 배치 샘플링을 하면서 지금 다 돌았는지 아닌지를 알 수 있으니까)

    def __getitem__(self, idx):
        # 이름에서 알 수 있듯이 데이터셋의 본분인 데이터 하나씩 뽑기이다. 
        # idx는 index를 말하는데, 몇 번째 데이터를 뽑을 건지에 대한 변수이다. 이는 데이터로더에서 또 사용될 것이다.
        # return으로 몇개의 데이터를 반환 할건지 정할 수 있다

밑바닥부터 시작하는 딥러닝2 의 CBOW에서 사용할 데이터셋을 만들어 보겠다!

In [6]:
# 텍스트를 넣으면 자동적으로 말뭉치와 word_to_id, id_to_word를 만들어 준다
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

# --------------------------------------------------------------------

def create_contexts_target(corpus, window_size=1):
    '''맥락과 타깃 생성

    :param corpus: 말뭉치(단어 ID 목록)
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return:
    '''
    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)

# --------------------------------------------------------------------

def convert_one_hot(corpus, vocab_size):
    '''원핫 표현으로 변환

    :param corpus: 단어 ID 목록(1차원 또는 2차원 넘파이 배열)
    :param vocab_size: 어휘 수
    :return: 원핫 표현(2차원 또는 3차원 넘파이 배열)
    '''
    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot    

우리가 만들 데이터셋은 다음과 같은 기능을 해아한다

1. 텍스트만 넣었는데 추론 기반 기법으로 학습할 수 있게끔 데이터들을 추출할 수 있어야 한다! 즉 다음과 같은 변환을 자동적으로 해줘야 한다.
</br> </br>

![1](./%EC%9D%B4%EB%AF%B8%EC%A7%80/image_20.png)



In [81]:
import torch

text = 'Yoy say goodbye and I say hello.'

class Corpus_Dataset(Dataset):
    def __init__(self, text, window_size = 1):
        self.window_size = window_size
        self.corpus, self.word_to_id, self.id_to_word = preprocess(text) # 먼저 텍스트를 말뭉치와 word_to_id로 바꾼다
        self.vocab_size = len(self.word_to_id)

        contexts, target = create_contexts_target(self.corpus, self.window_size)
        self.contexts = convert_one_hot(contexts, self.vocab_size)
        self.target = convert_one_hot(target, self.vocab_size)
        self.target = target

    def __len__(self):
        return len(self.contexts) # 데이터의 갯수 반환 형상이 (6,2,7)이면 데이터의 갯수는 6이다.

    def __getitem__(self, idx): # 데이터를 몇개씩 뽑을거냐 인데 저렇게 해두면 데이터가 올바르게 추출된다
        input = torch.tensor(self.contexts[idx], dtype = torch.float32)
        label = torch.tensor(self.target[idx], dtype = torch.long) # 손실함수 구할때 라벨은 long로 해야 한다고 하더라...
        return input, label

In [85]:
from torch.utils.data import DataLoader

dataset = corpus_Dataset(text)
dataLoader = DataLoader(dataset)

for input, label in dataLoader:
    print(input.shape)


torch.Size([1, 2, 7])
torch.Size([1, 2, 7])
torch.Size([1, 2, 7])
torch.Size([1, 2, 7])
torch.Size([1, 2, 7])
torch.Size([1, 2, 7])
